In [1]:
import os
import re
import boto3
import pandas as pd
from io import BytesIO
from acessar_dados_curated import carregar_parquets_em_variaveis

In [2]:
carregar_parquets_em_variaveis(prefix="silver/")

clientes_desde ← silver/clientes_desde.parquet  | 10615 linhas
contratacoes_ultimos_12_meses ← silver/contratacoes_ultimos_12_meses.parquet  | 4314 linhas
dados_clientes ← silver/dados_clientes.parquet  | 238597 linhas
historico ← silver/historico.parquet  | 22740 linhas
mrr ← silver/mrr.parquet  | 7309 linhas
nps_relacional ← silver/nps_relacional.parquet  | 13942 linhas
nps_transacional_aquisicao ← silver/nps_transacional_aquisicao.parquet  | 175 linhas
nps_transacional_implantacao ← silver/nps_transacional_implantacao.parquet  | 662 linhas
nps_transacional_onboarding ← silver/nps_transacional_onboarding.parquet  | 208 linhas
nps_transacional_produto ← silver/nps_transacional_produto.parquet  | 113201 linhas
nps_transacional_suporte ← silver/nps_transacional_suporte.parquet  | 74794 linhas
telemetria_consolidado ← silver/telemetria_consolidado.parquet  | 30804197 linhas
tickets ← silver/tickets.parquet  | 976063 linhas


{'clientes_desde': 's3://arcana-fiap/silver/clientes_desde.parquet',
 'contratacoes_ultimos_12_meses': 's3://arcana-fiap/silver/contratacoes_ultimos_12_meses.parquet',
 'dados_clientes': 's3://arcana-fiap/silver/dados_clientes.parquet',
 'historico': 's3://arcana-fiap/silver/historico.parquet',
 'mrr': 's3://arcana-fiap/silver/mrr.parquet',
 'nps_relacional': 's3://arcana-fiap/silver/nps_relacional.parquet',
 'nps_transacional_aquisicao': 's3://arcana-fiap/silver/nps_transacional_aquisicao.parquet',
 'nps_transacional_implantacao': 's3://arcana-fiap/silver/nps_transacional_implantacao.parquet',
 'nps_transacional_onboarding': 's3://arcana-fiap/silver/nps_transacional_onboarding.parquet',
 'nps_transacional_produto': 's3://arcana-fiap/silver/nps_transacional_produto.parquet',
 'nps_transacional_suporte': 's3://arcana-fiap/silver/nps_transacional_suporte.parquet',
 'telemetria_consolidado': 's3://arcana-fiap/silver/telemetria_consolidado.parquet',
 'tickets': 's3://arcana-fiap/silver/tic

In [3]:
df_clientes_desde = clientes_desde
df_contratacoes_ultimos_12_meses = contratacoes_ultimos_12_meses
df_dados_clientes = dados_clientes
df_historico = historico
df_mrr = mrr
df_nps_relacional = nps_relacional
df_nps_transacional_aquisicao = nps_transacional_aquisicao
df_nps_transacional_implantacao = nps_transacional_implantacao
df_nps_transacional_onboarding = nps_transacional_onboarding
df_nps_transacional_produto = nps_transacional_produto
df_nps_transacional_suporte = nps_transacional_suporte
df_telemetria_consolidado = telemetria_consolidado
df_tickets = tickets

NameError: name 'clientes_desde' is not defined

In [20]:
df_tickets.head()

,COD_CLIENTE,NOME_GRUPO,TIPO_TICKET,STATUS_TICKET,DATA_CRIACAO,DATA_ATUALIZACAO,BK_TICKET,PRIORIDADE_TICKET
0,TFCPWG,Fábrica Consinco,task,closed,2024-11-12,2025-01-02,21787048,low
1,TB2434,PC - Financeiro,question,closed,2024-05-07,2024-05-16,20066662,normal
2,TFBYNF,TOTVS Chef Corporativo,question,closed,2024-03-19,2024-04-03,19662250,low
3,TFCNAD,PC - Financeiro,question,closed,2024-06-25,2024-07-04,20517470,normal
4,TFDJLF,AGT CLOUD RM STD,question,closed,2024-02-02,2024-02-06,19222316,low
